# Preparing Batch File
Batches start with a .jsonl file where each line contains the details of an individual request t

In [ ]:
{"custom_id": "request-1",
 "method": "POST", "url": "/v1/chat/completions",
 "body": {
     "model": "gpt-3.5-turbo-0125", "messages": [
         {"role": "system", "content": "You are a helpful assistant."},
         {"role": "user", "content": "Hello world!"}],
     "max_tokens": 1000}}



# upload the .jsonl Batch file

In [ ]:
from openai import OpenAI
client = OpenAI()

batch_input_file = client.files.create(
    file=open("batchinput.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file)

# Creating the Batch
Once you've successfully uploaded your input file, you can use the input File object's ID to create a batch. In this case, let's assume the file ID is file-abc123. For now, the completion window can only be set to 24h. You can also provide custom metadata via an optional metadata parameter.

In [ ]:
from openai import OpenAI
client = OpenAI()

batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
)

this should be the output:
```python
{
  "id": "batch_abc123",
  "object": "batch",
  "endpoint": "/v1/chat/completions",
  "errors": null,
  "input_file_id": "file-abc123",
  "completion_window": "24h",
  "status": "validating",
  "output_file_id": null,
  "error_file_id": null,
  "created_at": 1714508499,
  "in_progress_at": null,
  "expires_at": 1714536634,
  "completed_at": null,
  "failed_at": null,
  "expired_at": null,
  "request_counts": {
    "total": 0,
    "completed": 0,
    "failed": 0
  },
  "metadata": null
}
````

# Checking the Status of a Batch

In [ ]:
from openai import OpenAI
client = OpenAI()

batch = client.batches.retrieve("batch_abc123")
print(batch)

### status can be:

Status	        Description
validating	    the input file is being validated before the batch can begin
failed	the     input file has failed the validation process
in_progress	    the input file was successfully validated and the batch is currently being run
finalizing	    the batch has completed and the results are being prepared
completed	    the batch has been completed and the results are ready
expired	the     batch was not able to be completed within the 24-hour time window
cancelling	    the batch is being cancelled (may take up to 10 minutes)
cancelled	    the batch was cancelled

#  Retrieving the Results

Note that the output line order may not match the input line order. Instead of relying on order to process your results, use the custom_id field which will be present in each line of your output file and allow you to map requests in your input to results in your output.

In [ ]:
from openai import OpenAI
client = OpenAI()

file_response = client.files.content("file-xyz123")
print(file_response.text)